# Task 2

Implement a circuit that returns |01> and |10> with equal probability (50% for each).

Requirements :

The circuit should consist only of CNOTs, RXs and RYs. 
Start from all parameters in parametric gates being equal to 0 or randomly chosen. 
You should find the right set of parameters using gradient descent (you can use more advanced optimization methods if you like). 
Simulations must be done with sampling (i.e. a limited number of measurements per iteration) and noise. 

Compare the results for different numbers of measurements: 1, 10, 100, 1000. 

Bonus question:
How to make sure you produce state |01> + |10> and not |01> - |10> ?

(Actually for more careful readers, the “correct” version of this question is posted below:
How to make sure you produce state  |01⟩  +  |10⟩  and not any other combination of |01> + e(i*phi)|10⟩ (for example |01⟩  -  |10⟩)?)

## Preliminar questions

Finding a circuit which solves the problem directily is not difficult. One initial attempt to approach the porposed problem might be the following:

- Starting from the structure of circuit which solves the problem, we can replace each gate with one more general that could be parametrized in order to, after that, search the valid solution via gradient descent. This solution doesn't seem to be easy to generalize nor even right. So I keept exploring.
- Reading the Michał Stęchły's blog and qiskit textbook about VQE, I found the way to create a universal parameterized 2 qubit circuit.

### Gradient descent

- We define a cost function which codify the difference between our output an the desired one, the entangled state.
- We are asked to do the simulations with sampling and noise, so we'll use normalized $\psi$, with $l_2$ norm, as probability distibutions to sample from. Then, we average the results over the samples and calculate the differences.
- We apply our universal gate operator $U(\theta_1, ..., \theta_n)$ to the ground state |00>.
- $\theta_j \leftarrow \theta_j - \alpha\cdot\frac{\partial}{\partial \theta_j}J(\theta_1, ..., \theta_n)$
- We need to calculate the partial derivatives with respect to the parameters. In our case, we need to work by sampling so we must to take care about the way we do that in torch.

In [1]:
import torch
import numpy as np
import itertools
import qiskit
from qiskit import QuantumCircuit, Aer, BasicAer, execute
from qiskit.visualization import plot_histogram
# from qiskit.textbook.tools import array_to_latex
import matplotlib.pyplot as plt
%matplotlib inline

### Plot the graph of the universal circuit following the reference.

In [2]:
θ_0 = np.random.rand(1)
ϕ_0 = np.random.rand(1)
λ_0 = np.random.rand(1)
θ_1 = np.random.rand(1)
ϕ_1 = np.random.rand(1)
λ_1 = np.random.rand(1)
θ_2 = np.random.rand(1)
ϕ_2 = np.random.rand(1)
λ_2 = np.random.rand(1)
θ_3 = np.random.rand(1)
ϕ_3 = np.random.rand(1)
λ_3 = np.random.rand(1)
θ_4 = np.random.rand(1)
ϕ_4 = np.random.rand(1)
λ_4 = np.random.rand(1)
θ_5 = np.random.rand(1)
ϕ_5 = np.random.rand(1)
λ_5 = np.random.rand(1)
θ_6 = np.random.rand(1)
ϕ_6 = np.random.rand(1)
λ_6 = np.random.rand(1)
θ_7 = np.random.rand(1)
ϕ_7 = np.random.rand(1)
λ_7 = np.random.rand(1)

qc = QuantumCircuit(2)
qc.u3(θ_0, ϕ_0, λ_0, 0)
qc.u3(θ_1, ϕ_1, λ_1, 1)
qc.cx(1, 0)
qc.u3(θ_2, ϕ_2, λ_2, 0)
qc.u3(θ_3, ϕ_3, λ_3, 1)
qc.cx(0, 1)
qc.u3(θ_4, ϕ_4, λ_4, 0)
qc.u3(θ_5, ϕ_5, λ_5, 1)
qc.cx(1, 0)
qc.u3(θ_6, ϕ_6, λ_6, 0)
qc.u3(θ_7, ϕ_7, λ_7, 1)
qc.cx(0, 1)
qc.draw()

┌────┐┌───┐┌────┐     ┌────┐┌───┐┌────┐     
q_0: ┤ U3 ├┤ X ├┤ U3 ├──■──┤ U3 ├┤ X ├┤ U3 ├──■──
     ├────┤└─┬─┘├────┤┌─┴─┐├────┤└─┬─┘├────┤┌─┴─┐
q_1: ┤ U3 ├──■──┤ U3 ├┤ X ├┤ U3 ├──■──┤ U3 ├┤ X ├
     └────┘     └────┘└───┘└────┘     └────┘└───┘

### Pytorch implementation of the universal circuit.

- We need the CNOT gates.
- We need U3.

In [3]:
cx_01 = torch.tensor([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])
cx_10 = torch.tensor([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

In [6]:
def torch_u3(θ, ϕ, λ, index):
    """
    Implements the universal parameterized 1 qubit circuit over one index of the tensor product space.
    
    :param θ: float defining the angle θ.
    :param ϕ: float defining the angle ϕ.
    :param λ: float defining the angle λ.
    :param index: integer which define the index to apply the rotation over.
    :return: Torch tensor as tensor product of identity and U3 gate.
    """
    j = torch.imag(torch.tensor(1, dtype=torch.complex64))
    identity = torch.tensor([[1.0, 0.0], [0.0, 1.0]])
    u3 = torch.tensor([[torch.cos(θ / 2), -1 * torch.exp(-1 * j * λ) * torch.sin(θ / 2)],
                       [torch.exp(j * ϕ) * torch.sin(θ / 2), torch.exp(j * (λ + ϕ)) * torch.cos(θ / 2)]])
    if index == 0:
        return -1 * kronecker(identity, u3)
    else:
        return -1 * kronecker(u3, identity)

### Kronecker product in Pytorch

In [13]:
# https://discuss.pytorch.org/t/kronecker-product/3919
def kronecker(A, B):
    return torch.einsum("ab,cd->acbd", A, B).view(A.size(0)*B.size(0),  A.size(1)*B.size(1))

### Sampling

In [5]:
def get_expected_value_by_sampling(ψ, num_samples):
    """Simulates the process of measurement returning the function ψ re-calculated over his own samples.
    
    :param ψ: Wave function which we want to measure
    :param num_samples: integer with the number of samples.
    :return: ψ recalculated.
    """
    counts = {k: 0 for k in range(len(ψ))}
    samples = sorted(get_torch_samples(num_samples, ψ))
    counts.update({k: len(list(g)) / num_samples for k, g in itertools.groupby(samples)})
    return torch.tensor(t

### Implementation in Pytorch of sampling via cumulative density function.

In [8]:
bin_edges = torch.tensor([0, 1, 2, 3])
def icdf(x, cdf):
    return int(bin_edges[torch.where(cdf >= x)][0])

In [9]:
def get_torch_samples(num_samples, ψ):
    if torch.cumsum(ψ**2, 0)[-1] != 1.0:
        ψ = torch.sqrt(ψ**2 / torch.sum(ψ**2))
    u = torch.rand(num_samples)
    return [icdf(x, torch.cumsum(ψ**2, 0)) for x in u]

### Gradient Descent

In [10]:
num_samples = 1000
num_iterations = 400

θ_0 = torch.randn(1, requires_grad=True)
ϕ_0 = torch.randn(1, requires_grad=True)
λ_0 = torch.randn(1, requires_grad=True)
θ_1 = torch.randn(1, requires_grad=True)
ϕ_1 = torch.randn(1, requires_grad=True)
λ_1 = torch.randn(1, requires_grad=True)
θ_2 = torch.randn(1, requires_grad=True)
ϕ_2 = torch.randn(1, requires_grad=True)
λ_2 = torch.randn(1, requires_grad=True)
θ_3 = torch.randn(1, requires_grad=True)
ϕ_3 = torch.randn(1, requires_grad=True)
λ_3 = torch.randn(1, requires_grad=True)
θ_4 = torch.randn(1, requires_grad=True)
ϕ_4 = torch.randn(1, requires_grad=True)
λ_4 = torch.randn(1, requires_grad=True)
θ_5 = torch.randn(1, requires_grad=True)
ϕ_5 = torch.randn(1, requires_grad=True)
λ_5 = torch.randn(1, requires_grad=True)
θ_6 = torch.randn(1, requires_grad=True)
ϕ_6 = torch.randn(1, requires_grad=True)
λ_6 = torch.randn(1, requires_grad=True)
θ_7 = torch.randn(1, requires_grad=True)
ϕ_7 = torch.randn(1, requires_grad=True)
λ_7 = torch.randn(1, requires_grad=True)

ψ_target = torch.tensor([0.0, 1/np.sqrt(2), -1/np.sqrt(2), 0.0])

qc = torch.tensor(
    torch_u3(θ_0, ϕ_0, λ_0, 0) *
    torch_u3(θ_1, ϕ_1, λ_1, 1) *
    cx_10 *
    torch_u3(θ_2, ϕ_2, λ_2, 0) *
    torch_u3(θ_3, ϕ_3, λ_3, 1) *
    cx_01 *
    torch_u3(θ_4, ϕ_4, λ_4, 0) *
    torch_u3(θ_5, ϕ_5, λ_5, 1) *
    cx_10 *
    torch_u3(θ_6, ϕ_6, λ_6, 0) *
    torch_u3(θ_7, ϕ_7, λ_7, 1) *
    cx_01, requires_grad=True)
ψ_pred = torch.rand(4, requires_grad=True)
ψ_pred = torch.sqrt(ψ_pred**2 / torch.sum(ψ_pred**2))
cost_function = torch.sum((ψ_target - ψ_pred) ** 2)

gamma = 0.01
for i in range(num_iterations):
    ψ_pred = get_expected_value_by_sampling(ψ_pred, num_samples)
    ψ_pred = torch.sqrt(ψ_pred**2 / torch.sum(ψ_pred**2))
    ψ_pred = torch.matmul(qc, ψ_pred)

    cost_function = torch.sum((ψ_target - ψ_pred) ** 2)

    # backward
    if i < 499:
        cost_function.backward(retain_graph=True)
    else:
        cost_function.backward()
    print('Iteration {}, cost_function: {}'.format(i, cost_function))
    with torch.no_grad():
        qc = qc - gamma * qc.grad
    qc.requires_grad = True


<ipython-input-10-3ccbb8479289>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  qc = torch.tensor(
<ipython-input-5-013b40f5e5fb>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(torch.sqrt(torch.tensor([v for v in counts.values()])), requires_grad=True)


Iteration 0, cost_function: 1.2315635739944673
Iteration 1, cost_function: 1.4763765367540644
Iteration 2, cost_function: 1.4179120485358325
Iteration 3, cost_function: 1.3578003116544435
Iteration 4, cost_function: 1.3037791742236653
Iteration 5, cost_function: 1.251400560314594
Iteration 6, cost_function: 1.1947573537415073
Iteration 7, cost_function: 1.1441059274415597
Iteration 8, cost_function: 1.0968803801300304
Iteration 9, cost_function: 1.0466829425790682
Iteration 10, cost_function: 0.9985827371418319
Iteration 11, cost_function: 0.9585310257084343
Iteration 12, cost_function: 0.9082878837860267
Iteration 13, cost_function: 0.8637804493030541
Iteration 14, cost_function: 0.8262753370307041
Iteration 15, cost_function: 0.7873251887748725
Iteration 16, cost_function: 0.7524526128566997
Iteration 17, cost_function: 0.7155522556703439
Iteration 18, cost_function: 0.681813990840573
Iteration 19, cost_function: 0.6540590285980928
Iteration 20, cost_function: 0.6196410734250576
Iter

In [11]:
ψ_pred

tensor([-4.1742e-05,  7.0716e-01, -7.0716e-01,  0.0000e+00],
       grad_fn=<MvBackward>)

In [12]:
ψ_target

tensor([ 0.0000,  0.7071, -0.7071,  0.0000], dtype=torch.float64)

# Bonus question.

I think we would need to change the measurement basis because both states are indistinguishable on the current one.